<a href="https://colab.research.google.com/github/johntanas/it1244project/blob/main/code/model_no_std.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
path_to_evm_label="/content/expo_moving_k2_labels.csv" #https://drive.google.com/file/d/1RUSdg3mKkoY984r4s7rGabjwhJEcIA_2/view?usp=sharing
path_to_app="/content/cleaned_application.csv" #https://drive.google.com/file/d/1Ktdv2NTyK2AM48yBvbpFezd82-srqj7w/view?usp=sharing
onlinedataset=True
if onlinedataset:
  url = "https://drive.google.com/file/d/1Ktdv2NTyK2AM48yBvbpFezd82-srqj7w/view?usp=sharing"
  url='https://drive.google.com/uc?id=' + url.split('/')[-2]
  cleaned_app_df=pd.read_csv(url,index_col=0)
  url = "https://drive.google.com/file/d/1RUSdg3mKkoY984r4s7rGabjwhJEcIA_2/view?usp=sharing"
  url='https://drive.google.com/uc?id=' + url.split('/')[-2]
  ewm_df_status=pd.read_csv(url,index_col=0)
else:
  cleaned_app_df=pd.read_csv(path_to_app,index_col=0)
  ewm_df_status=pd.read_csv(path_to_evm_label,index_col=0)

In [ ]:
labels_to_int_evm={"bad":0,"good":1}

In [ ]:
ewm_df=ewm_df_status.drop(columns="status").replace({"label":labels_to_int_evm})

In [ ]:
dep_var="label"
random_state=42

In [ ]:
from sklearn.model_selection import ShuffleSplit
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier,export_text,plot_tree
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay

In [ ]:
df1=ewm_df.merge(cleaned_app_df,how="inner",on="id").drop("id",axis=1)

In [ ]:
df=df1.replace(to_replace = ("Y", "N"),
             value = (1, 0))

In [ ]:
gender = pd.get_dummies(df["gender"])
income_type = pd.get_dummies(df["income_type"]).drop("Pensioner",axis=1)
education_level = pd.get_dummies(df["education_level"])
family_status = pd.get_dummies(df["family_status"])
house_type = pd.get_dummies(df["house_type"])
job = pd.get_dummies(df["job"])
cont=df[["income","birth_day",	"employment_length"]]
one_hot=df.drop(columns = ["gender", "income_type", "education_level",
                   "family_status", "house_type", "job"])


# Standardizing the training and test data
from sklearn.preprocessing import StandardScaler
std_x = StandardScaler()
std_x.fit(cont)
cont_std = std_x.transform(cont)

In [ ]:
one_hot = pd.concat([pd.DataFrame(cont_std,columns=["income","birth_day",	"employment_length"]), gender, income_type, education_level, family_status,
                house_type, job,df["label"]], axis = 1)

In [ ]:
yvals = pd.DataFrame(one_hot["label"])
xvals = one_hot.drop(columns = "label")

In [ ]:
from sklearn.model_selection import train_test_split
xs, valid_xs, y, valid_y = train_test_split(xvals, yvals, test_size = 0.20,
                                                random_state = 42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
int_to_labels_evm={"bad":0,"good":1}

https://www.kaggle.com/code/umerkk12/credit-card-predictive-analysis#there-are-9516-rows.

In [ ]:
classifiers = {
    "LogisticRegression" : LogisticRegression(),
    "KNeighbors" : KNeighborsClassifier(),
    "SVC" : SVC(),
    "DecisionTree" : DecisionTreeClassifier(),
    "RandomForest" : RandomForestClassifier(),
    "XGBoost" : XGBClassifier()
}

In [ ]:
for key, classifier in classifiers.items():
    classifier.fit(xs, y)
    prediction = classifier.predict(valid_xs)
    print("Model "+key+ " using unbalanced data")
    print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

Model LogisticRegression using unbalanced data
              precision    recall  f1-score   support

         bad       1.00      0.00      0.00       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro avg       0.86      0.50      0.42      1513
weighted avg       0.80      0.73      0.62      1513

Model KNeighbors using unbalanced data
              precision    recall  f1-score   support

         bad       0.32      0.13      0.19       410
        good       0.73      0.89      0.81      1103

    accuracy                           0.69      1513
   macro avg       0.53      0.51      0.50      1513
weighted avg       0.62      0.69      0.64      1513

Model SVC using unbalanced data
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro a

In [ ]:
import keras
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten


In [ ]:
one_hot_y=pd.get_dummies(y["label"])
one_hot_valid_y=pd.get_dummies(valid_y["label"])

In [ ]:
model=keras.Sequential([
keras.Input(shape=(44,)),
keras.layers.Dense(1024),
keras.layers.Activation("sigmoid"),
keras.layers.Dense(512),
keras.layers.Activation(activation="relu"),
keras.layers.Dense(2),
keras.layers.Activation(activation="softmax")])

use BinaryFocalCrossentropy for imbalanced data

In [ ]:
opt = tf.optimizers.SGD(learning_rate=0.01)
model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(), optimizer=opt, metrics=['accuracy',tf.keras.metrics.Recall()])
epochs = 10
history=model.fit(xs, one_hot_y,batch_size=32, epochs=epochs,validation_data=(valid_xs,one_hot_valid_y))

Epoch 1/10
190/190 [==============================] - 4s 14ms/step - loss: 0.1525 - accuracy: 0.7145 - recall: 0.7145 - val_loss: 0.1944 - val_accuracy: 0.7290 - val_recall: 0.7290
Epoch 2/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1538 - accuracy: 0.7197 - recall: 0.7197 - val_loss: 0.1887 - val_accuracy: 0.7290 - val_recall: 0.7290
Epoch 3/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1525 - accuracy: 0.7215 - recall: 0.7215 - val_loss: 0.1868 - val_accuracy: 0.7290 - val_recall: 0.7290
Epoch 4/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1510 - accuracy: 0.7238 - recall: 0.7238 - val_loss: 0.1803 - val_accuracy: 0.2981 - val_recall: 0.2981
Epoch 5/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1517 - accuracy: 0.7228 - recall: 0.7228 - val_loss: 0.1877 - val_accuracy: 0.2730 - val_recall: 0.2730
Epoch 6/10
190/190 [==============================] - 2s 11ms/step - loss: 0.1523 - accuracy: 0

In [ ]:
y_prob=model.predict(valid_xs)
prediction = y_prob.argmax(axis=-1)
print("Model Neural Net")
print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

48/48 [==============================] - 0s 3ms/step
Model Neural Net
              precision    recall  f1-score   support

         bad       0.27      1.00      0.43       410
        good       0.00      0.00      0.00      1103

    accuracy                           0.27      1513
   macro avg       0.14      0.50      0.21      1513
weighted avg       0.07      0.27      0.12      1513



In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_balanced, y_balanced = oversample.fit_resample(xs, y)

In [ ]:
for key, classifier in classifiers.items():
    classifier.fit(X_balanced, y_balanced)
    prediction = classifier.predict(valid_xs)
    print("Model "+key)
    print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

Model LogisticRegression
              precision    recall  f1-score   support

         bad       0.29      0.20      0.23       410
        good       0.73      0.82      0.77      1103

    accuracy                           0.65      1513
   macro avg       0.51      0.51      0.50      1513
weighted avg       0.61      0.65      0.63      1513

Model KNeighbors
              precision    recall  f1-score   support

         bad       0.27      0.43      0.33       410
        good       0.73      0.57      0.64      1103

    accuracy                           0.54      1513
   macro avg       0.50      0.50      0.49      1513
weighted avg       0.61      0.54      0.56      1513

Model SVC
              precision    recall  f1-score   support

         bad       0.27      0.23      0.25       410
        good       0.73      0.77      0.75      1103

    accuracy                           0.63      1513
   macro avg       0.50      0.50      0.50      1513
weighted avg       0.6

In [ ]:
one_hot_y_balanced=pd.get_dummies(y_balanced["label"])

In [ ]:
model2=keras.Sequential([
keras.Input(shape=(44,)),
keras.layers.Dense(1024),
keras.layers.Activation("sigmoid"),
keras.layers.Dense(512),
keras.layers.Activation(activation="relu"),
keras.layers.Dense(2),
keras.layers.Activation(activation="softmax")])

Use BinaryCrossentropy instead

In [ ]:
opt = tf.optimizers.SGD(learning_rate=0.01)
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy',tf.keras.metrics.Recall()])
epochs = 10
history=model2.fit(X_balanced,one_hot_y_balanced,batch_size=32, epochs=epochs,validation_data=(valid_xs,one_hot_valid_y))

Epoch 1/10
275/275 [==============================] - 4s 12ms/step - loss: 0.6968 - accuracy: 0.5020 - recall_1: 0.5020 - val_loss: 0.7137 - val_accuracy: 0.2710 - val_recall_1: 0.2710
Epoch 2/10
275/275 [==============================] - 3s 11ms/step - loss: 0.6956 - accuracy: 0.5019 - recall_1: 0.5019 - val_loss: 0.6564 - val_accuracy: 0.7290 - val_recall_1: 0.7290
Epoch 3/10
275/275 [==============================] - 3s 11ms/step - loss: 0.6968 - accuracy: 0.4941 - recall_1: 0.4941 - val_loss: 0.6643 - val_accuracy: 0.7297 - val_recall_1: 0.7297
Epoch 4/10
275/275 [==============================] - 3s 10ms/step - loss: 0.6968 - accuracy: 0.5019 - recall_1: 0.5019 - val_loss: 0.6512 - val_accuracy: 0.7297 - val_recall_1: 0.7297
Epoch 5/10
275/275 [==============================] - 3s 11ms/step - loss: 0.6957 - accuracy: 0.5014 - recall_1: 0.5014 - val_loss: 0.6714 - val_accuracy: 0.7297 - val_recall_1: 0.7297
Epoch 6/10
275/275 [==============================] - 3s 10ms/step - loss: 

In [ ]:
y_prob=model2.predict(valid_xs)
prediction = y_prob.argmax(axis=-1)
print("Model Neural Net")
print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

48/48 [==============================] - 0s 3ms/step
Model Neural Net
              precision    recall  f1-score   support

         bad       0.27      1.00      0.43       410
        good       0.00      0.00      0.00      1103

    accuracy                           0.27      1513
   macro avg       0.14      0.50      0.21      1513
weighted avg       0.07      0.27      0.12      1513

